In [2]:

!pip install --upgrade \
    transformers==4.43.0 \
    sentence-transformers==2.6.1 \
    accelerate==0.29.3 \
    bitsandbytes==0.43.1

!pip install faiss-cpu==1.8.0
!pip install chromadb==0.5.3


!pip install langchain==0.2.0
!pip install pypdf==4.2.0
!pip install rapidfuzz==3.6.2


!pip install fastapi==0.116.1.0 uvicorn==0.30.1


!pip install tqdm==4.66.4


ERROR: Ignored the following yanked versions: 0.17.0
ERROR: Could not find a version that satisfies the requirement uvicorn==0.35.1 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.8, 0.2.9, 0.2.10, 0.2.11, 0.2.12, 0.2.13, 0.2.14, 0.2.15, 0.2.16, 0.2.17, 0.2.18, 0.2.19, 0.2.20, 0.2.21, 0.2.22, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.6, 0.3.7, 0.3.8, 0.3.9, 0.3.10, 0.3.11, 0.3.12, 0.3.13, 0.3.14, 0.3.15, 0.3.16, 0.3.17, 0.3.18, 0.3.19, 0.3.20, 0.3.21, 0.3.22, 0.3.23, 0.3.24, 0.3.25, 0.3.26, 0.3.27, 0.3.28, 0.3.29, 0.3.30, 0.3.31, 0.3.32, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 0.6.1, 0.7.0b1, 0.7.0b2, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.10.2, 0.10.3, 0.10.4, 0.10.5, 0.10.6, 0.10.7, 0.10.8, 0.10.9, 0.11.0, 0.11.1, 0.11.2, 0.

In [5]:
!pip install -U langchain-community


  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.0/368.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
  Attempting uninstall: langchain-

In [2]:
from langchain.document_loaders import PyPDFLoader, TextLoader
loader = PyPDFLoader("http://facweb.cs.depaul.edu/sjost/it211/documents/think-python-2nd.pdf#page=50.20")       # each page = Document
docs   = loader.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " "],
    chunk_size=450,
    chunk_overlap=60
)

chunks = splitter.split_documents(docs)

from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

import faiss
import numpy as np

embs = encoder.encode([c.page_content for c in chunks], show_progress_bar=True)

index = faiss.IndexFlatIP(embs.shape[1])   # cosine similarity
index.add(np.array(embs, dtype='float32'))

def get_top_k(query, k=4):
    q_emb = encoder.encode([query])
    D, I = index.search(np.array(q_emb, dtype='float32'), k)
    return [chunks[i] for i in I[0]]

def build_prompt(query, docs):
    context = "\n\n".join(
        f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)
    )
    prompt = (
        "You are an expert assistant.\n"
        "Answer the question using ONLY the sources provided.\n\n"
        f"{context}\n\n"
        f"Question: {query}\n"
        "Answer:"
    )
    return prompt

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = 'google/flan-t5-base'
tok   = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_answer(query, k=4, max_tokens=256):
    docs  = get_top_k(query, k)
    prompt = build_prompt(query, docs)
    input_ids = tok(prompt, return_tensors='pt').input_ids
    out = model.generate(
        input_ids, max_new_tokens=max_tokens,
        temperature=0.2, top_p=0.9
    )
    return tok.decode(out[0], skip_special_tokens=True)


if __name__ == "__main__":
    while True:
        q = input("\nUser: ")
        if q.lower() in {"exit", "quit"}:
            break
        print("\nBot:", generate_answer(q))

from fastapi import FastAPI, Body
app = FastAPI()

@app.post("/ask")
def ask(q: str = Body()):
    return {"answer": generate_answer(q)}


/usr/local/lib/python3.11/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


User: How do I configure the database connection?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Bot: >>> import dbm >>> db = dbm.open('captions', 'c') The mode 'c' means that the database should be created if it doesn’t already exist. The result is a database object that can be used (for most operations) like a dictionary. The module dbm provides an interface for creating and updating database files. As with other files, you should close the database when you are done: >>> db.close() Pickling

User: Why is Python best for Deep Learning and Gen AI?

Bot: xii | Preface

User: Explain for loop.

Bot: Source 1: total = 0 for x in t: total += x return total total is initialized to 0 each time through the loop, x gets one element from the list. The += operator provides a short way to update a variable. This augmented assign ment statement, total += x is equivalent to total = total + x is called a loop because the body of the loop should change the value of one or more variables so that the condition becomes false eventually and the loop terminates. Source 2: This type of flow is calle